In [1]:
%pip install azureml-widgets
%pip install azureml-sdk[notebooks]
%pip install azureml-train-automl-runtime==1.57.0

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 11, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 53.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 64.5 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.4/121.4 KB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.4/617.4 KB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.9/143.9 KB 28.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.4
    Not uninstalling urllib3 at /home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages, outside environment /nfs4/pyenv-07c10438-e576-4f7f-b4a4-c66f52011e54
    Can't uninstall 'urllib3'. No files were found to uninstall.
  Attempting uninstall: isodate
    Found existing installation: isodate 0.6.0
    Not uninstalling isodate at /home/trusted-service-user/cluster-env/env/lib/python3.10/site-packages

In [2]:
from azureml.core import Workspace, Experiment

# ws = Workspace.from_config()
subscription_name = "quick-starts-ws-267001"
subscription_id='81cefad3-d2c9-4f77-a466-99a7f541c7bb' #61c5c3f0-6dc7-4ed9-a7f3-c704b20e3b30



ws = Workspace.get(name = subscription_name,
subscription_id=subscription_id,
resource_group="aml-quickstarts-267001"
)


exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')


run = exp.start_logging()

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 17, Finished, Available, Finished)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code BK2C9H2WP to authenticate.
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-267001
Azure region: southcentralus
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-267001


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = subscription_name

vm_size = "Standard_D2_V2"
compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size, max_nodes=4)
#Create the compute cluster
compute_target = ComputeTarget.create(ws,cluster_name, compute_config)
#wait for the compute ready
compute_target.wait_for_completion(show_output=True)

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 18, Finished, Available, Finished)

InProgress..
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [4]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {"C": uniform(0.05, 0.1),
    "max_iter" : choice(16, 32, 64, 128)}
)

# Specify a Policy
policy = BanditPolicy(slack_factor=0.2, evaluation_interval=1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', 
file_path='Users/odl_user_267001/conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(
    source_directory=".",
    script="Users/odl_user_267001/train.py",
    environment=sklearn_env,
    compute_target=compute_target
)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
    run_config=src,
    hyperparameter_sampling=ps,
    policy=policy,
    primary_metric_name="Accuracy",
    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
    max_total_runs=10,
    max_concurrent_runs=4
)

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 19, Finished, Available, Finished)

In [5]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

experiment = Experiment(workspace=ws, name="my-hyperdive-experiment")
experiment_run = experiment.submit(hyperdrive_config)
RunDetails(experiment_run).show()
experiment_run.wait_for_completion(show_output=True)

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 20, Finished, Available, Finished)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_9399fe68-32d0-40dd-87ee-706ddfbc247b
Web View: https://ml.azure.com/runs/HD_9399fe68-32d0-40dd-87ee-706ddfbc247b?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-267001/workspaces/quick-starts-ws-267001&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2024-08-24T13:59:33.724076][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2024-08-24T13:59:34.3112764Z][SCHEDULER][INFO]Scheduling job, id='HD_9399fe68-32d0-40dd-87ee-706ddfbc247b_0' 
[2024-08-24T13:59:34.5118431Z][SCHEDULER][INFO]Scheduling job, id='HD_9399fe68-32d0-40dd-87ee-706ddfbc247b_2' 
[2024-08-24T13:59:34.4502749Z][SCHEDULER][INFO]Scheduling job, id='HD_9399fe68-32d0-40dd-87ee-706ddfbc247b_1' 
[2024-08-24T13:59:34.6268641Z][SCHEDULER][INFO]Scheduling job, id='HD_9399fe68-32d0-40dd-87ee-706ddfbc247b_3' 
[2024-08-24T13:59:34.579996][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution t

{'runId': 'HD_9399fe68-32d0-40dd-87ee-706ddfbc247b',
 'target': 'quick-starts-ws-267001',
 'status': 'Completed',
 'startTimeUtc': '2024-08-24T13:59:32.673208Z',
 'endTimeUtc': '2024-08-24T14:19:19.331235Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name":"Accuracy","goal":"maximize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': '2ecf31a9-b81a-44d4-b200-9c0ef7593d85',
  'user_agent': 'python/3.10.6 (Linux-4.15.0-1178-azure-x86_64-with-glibc2.27) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.57.0',
  'space_size': 'infinite_space_size',
  'best_child_run_id': 'HD_9399fe68-32d0-40dd-87ee-706ddfbc247b_4',
  'score': '0.9083459787556905',
  'best_metric_status': 'Succeeded',
  'best_data_container_id': 'dcid.HD_9399fe68-32d0-40dd-87ee-706ddfbc247b_4'},
 'inputDatasets': [],
 'outputDatasets': [],
 'runDefinition': {'configuration': None,
  'attribution': None,
  '

In [6]:
all_runs = experiment.get_runs()
for run in all_runs:
    print(run.id, run.status)

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 21, Finished, Available, Finished)

HD_9399fe68-32d0-40dd-87ee-706ddfbc247b Completed


In [7]:
import joblib
# Get your best run and save the model from that run.

best_run = experiment_run.get_best_run_by_primary_metric()
print(best_run.get_metrics())

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 22, Finished, Available, Finished)

{'Regularization Strength:': 0.09671575274951592, 'Max iterations:': 64, 'Accuracy': 0.9083459787556905}


In [8]:
best_run.get_file_names()

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 23, Finished, Available, Finished)

['logs/azureml/dataprep/0/backgroundProcess.log',
 'logs/azureml/dataprep/0/backgroundProcess_Telemetry.log',
 'logs/azureml/dataprep/0/rslex.log.2024-08-24-14',
 'outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [9]:
#Save model
model = best_run.register_model(model_name = "auto_hyper_best_model",
model_path="outputs/model.joblib")

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 24, Finished, Available, Finished)

In [10]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_url = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(data_url)

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 25, Finished, Available, Finished)

In [13]:
from Users.odl_user_267001.train import clean_data

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 28, Finished, Available, Finished)

In [23]:
# from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 38, Finished, Available, Finished)

{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': 'cf3ea2c6-a860-450f-adf1-ce085f91bfed'}
{'infer_column_types': 'False', 'activity': 'to_pandas_dataframe', 'activityApp': 'TabularDataset', 'runId': 'cf3ea2c6-a860-450f-adf1-ce085f91bfed', 'run_id': 'cf3ea2c6-a860-450f-adf1-ce085f91bfed'}


In [24]:
from sklearn.model_selection import train_test_split
import pandas as pd

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 39, Finished, Available, Finished)

In [16]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
y_train = y_train.to_frame()

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 31, Finished, Available, Finished)

In [17]:
y_train.head()

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 32, Finished, Available, Finished)

,y
26465,0
1387,0
22628,0
8336,0
19898,0


In [19]:
training_data = pd.concat([x_train, y_train], axis=1)

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 34, Finished, Available, Finished)

In [20]:
training_data.head(3)

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 35, Finished, Available, Finished)

,age,marital,default,housing,loan,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,job_admin.,job_blue-collar,job_entrepreneur,job_housemaid,job_management,job_retired,job_self-employed,job_services,job_student,job_technician,job_unemployed,job_unknown,contact_cellular,contact_telephone,education_basic.4y,education_basic.6y,education_basic.9y,education_high.school,education_illiterate,education_professional.course,education_university.degree,education_unknown,y
26465,52,1,0,1,0,8,3,13,14,999,0,0,1.4,93.444,-36.1,4.964,5228.1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0
1387,41,1,0,1,0,6,4,84,3,999,0,0,1.4,94.465,-41.8,4.961,5228.1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0
22628,27,0,0,0,0,7,1,331,2,999,0,0,1.4,93.918,-42.7,4.962,5228.1,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0


In [29]:
# tabular_dataset = TabularDatasetFactory.register_pandas_dataframe(
#     dataframe=training_data,
#     target=ws.get_default_datastore(),
#     name="my_training_data"
# )

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 44, Finished, Available, Finished)

In [31]:
datastore = ws.get_default_datastore()
datastore

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 46, Finished, Available, Finished)

{
  "name": "workspaceblobstore",
  "container_name": "azureml-blobstore-17eb7207-7441-48eb-b3f0-4f14b5b3c7f0",
  "account_name": "mlstrg267001",
  "protocol": "https",
  "endpoint": "core.windows.net"
}

In [32]:
training_data.to_csv("Users/odl_user_267001/training_data.csv", index=False)

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 47, Finished, Available, Finished)

In [33]:
from azureml.core import Dataset

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 48, Finished, Available, Finished)

In [35]:
datastore.upload(src_dir='Users/odl_user_267001', target_path='data/')

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 50, Finished, Available, Finished)

"Datastore.upload" is deprecated after version 1.0.69. Please use "Dataset.File.upload_directory" to upload your files             from a local directory and create FileDataset in single method call. See Dataset API change notice at https://aka.ms/dataset-deprecation.
Uploading an estimated of 15 files
Uploading Users/odl_user_267001/.amlignore
Uploaded Users/odl_user_267001/.amlignore, 1 files out of an estimated total of 15
Uploading Users/odl_user_267001/.amlignore.amltmp
Uploaded Users/odl_user_267001/.amlignore.amltmp, 2 files out of an estimated total of 15
Uploading Users/odl_user_267001/conda_dependencies.yml
Uploaded Users/odl_user_267001/conda_dependencies.yml, 3 files out of an estimated total of 15
Uploading Users/odl_user_267001/train.py
Uploaded Users/odl_user_267001/train.py, 4 files out of an estimated total of 15
Uploading Users/odl_user_267001/train.py.amltmp
Uploaded Users/odl_user_267001/train.py.amltmp, 5 files out of an estimated total of 15
Uploading Users/odl_us

$AZUREML_DATAREFERENCE_91c34c41e95d42e199d233e273a97c75

In [36]:
my_data = Dataset.Tabular.from_delimited_files(path=[(datastore, ('data/training_data.csv'))])

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 51, Finished, Available, Finished)

In [37]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task="classification",
    primary_metric="accuracy",
    training_data=my_data,
    label_column_name='y',
    n_cross_validations=2,
    compute_target=compute_target)

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 52, Finished, Available, Finished)

In [38]:
# Submit your automl run

experiment_auto = Experiment(ws, 'automl_exp')
auto_run = experiment_auto.submit(automl_config, show_output=True)
RunDetails(auto_run).show()
auto_run.wait_for_completion(show_output=True)

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 53, Finished, Available, Finished)

Submitting remote run.
No run_configuration provided, running on quick-starts-ws-267001 with default configuration
Running on remote compute: quick-starts-ws-267001


Experiment,Id,Type,Status,Details Page,Docs Page
automl_exp,AutoML_a0cdf310-869c-4040-bae9-dbd076fdf34e,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+-------------------------------------

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl_exp,AutoML_a0cdf310-869c-4040-bae9-dbd076fdf34e,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+------------------------------+--------------------------------+--------------------------------------+
|Size of the smallest class    |Name/Label of the smallest class|Number of samples in the training data|
+==============================+================================+======================================+
|2951                          |True                            |26360                                 |
+------------------------------+--------------------------------+----

{'runId': 'AutoML_a0cdf310-869c-4040-bae9-dbd076fdf34e',
 'target': 'quick-starts-ws-267001',
 'status': 'Completed',
 'startTimeUtc': '2024-08-24T14:58:00.63819Z',
 'endTimeUtc': '2024-08-24T15:42:24.162141Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'quick-starts-ws-267001',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_exp","subscription_id":"81cefad3-d2c9-4f77-a466-99a7f541c7bb","resource_group":"aml-quickstarts-267001","workspace_name":"quick-starts-ws-267001","region":"southcentralus","compute_target"

In [42]:
# %pip install azureml-train-automl-runtime==1.57.0

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 61, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 420.7/420.7 KB 9.9 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 63.7 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.7/294.7 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 52.7 MB/s eta 0:00:0000:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.8/219.8 KB 37.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.1/38.1 MB 37.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.8/131.8 KB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.5/18.5 MB 43.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 52.1 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 53.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.5/182.5 KB 34.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32

In [43]:
# Retrieve and save your best automl model.

best_auto_run, best_auto_model = auto_run.get_output()

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 67, Finished, Available, Finished)

NameError: name 'auto_run' is not defined

In [44]:
auto_run

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 68, Finished, Available, Finished)

NameError: name 'auto_run' is not defined

In [41]:
best_auto_model

StatementMeta(f0fba55f-dd58-4613-82a0-22c2c1f74818, 0, 56, Finished, Available, Finished)